# Predicting home prices in Ames, IA

The purpose of this notebook is for me to learn something about Python programming, how to process data using Python, and how to perform machine learning tasks in Python. It may perhaps also provide some decent predictions of house prices in Ames, IA.

This data holds quite a bit of interest for me, because my wife and I are currently going through the process on both sides of the table - we've put our current house on the market, and we have a new house under contract.

I'd also like to invite anyone who takes the time to read this to point out ways in which my methods or code could be better, as I'm still figuring all this out. Also, **this is a work in progress**, so it will have many updates.

## 1 Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sb
from sklearn import linear_model
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor

## 2 Read in the data

In [2]:
train = pd.read_csv("./Data/train.csv", header=0)
test = pd.read_csv("./Data/test.csv", header=0)

In [3]:
train.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


## 3 Data cleaning
Let's check to what extent we need to clean up the data before throwing a model at it. On a previous version I had my own definition of these, but the results ended up being terrible, so I shamelessly stole large portions from [this kernel](https://www.kaggle.com/mountaindata/house-prices-advanced-regression-techniques/house-price-regression).

### 3.1 Get rid of NaNs
It looks like several columns have a lot of missing values. From the data dictionary provided, it seems that this should be expected. These should be interpreted as the absence of a given feature rather than a missing value. However, we're going to have to do something with them, because scikit-learn doesn't like them. We'll convert them to 0s.

In [4]:
train = train.fillna(0)
test = test.fillna(0)

### 3.2 Separating types of features
We want to get lists of all the features - one grand list, and also separate lists for categorical and numerical features. The former we will convert to Pandas's category type, and the latter we will leave alone. We won't carry the Id column (which is useless except as an identifier) and the sale price (the target). The middle set I've identified as "ordered categories", and I'll code them myself in what I feel is a natural way.

In [13]:
features = [x for x in train.columns if x not in ['id','SalePrice']]

In [14]:
cat_features = ['MSSubClass', 'MSZoning', 'Street','Alley', 
'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
'HouseStyle','OverallQual','OverallCond',
'Foundation', 'BsmtFinType1','BsmtFinType2', 
'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',
'Heating','CentralAir','Electrical','BsmtFullBath',
'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
'TotRmsAbvGrd','Functional','Fireplaces','GarageType',
'GarageFinish','GarageCars','PavedDrive',
'Fence','MiscFeature','SaleType','SaleCondition']

In [7]:
ord_cat_features = ['ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
                   'HeatingQC','KitchenQual','FireplaceQu','GarageQual','GarageCond','PoolQC']

In [8]:
num_features = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
'BsmtUnfSF','TotalBsmtSF', 'LowQualFinSF','1stFlrSF','2ndFlrSF',
'GrLivArea', 'YearRemodAdd', 'YearBuilt','GarageYrBlt','GarageArea','WoodDeckSF',
'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']

### 3.3 Converting categorical values to numeric values
We need to get rid of all the string values and replace them with numeric values. For this we'll lean on Pandas's categorical type, which didn't exist the last time I messed with Python extensively. They're a bit like R's "factors", which come in handy.

In [9]:
rows_train = len(train)
rows_test = len(test)

We'll now glue the training and test set together, for all but the sale price, which the test set doesn't have in this case. Later, we'll split them back apart. We do this because the test data might have instances within a category not seen in the test data, in which case Pandas won't pick them up. For example, if there are no values of 'I' in the 'MSZoning' field in the training data but not the test data, they'll be ignored when we convert the categories to codes. Gluing the datasets together prevents this. It wouldn't help at all if you had to run the model against data you didn't have access to (which is a possibility in this case).

Another thing that annoys me is that some of these categoricals could justifiably be considered to be ordered, in which case I want the order to be preserved. Pandas won't figure this out automatically. Some are easy, like OverallQual - which is already numbered from 1-10. Nothing more needs to be done. Others have 'excellent', 'good', etc., and we'd like these to mean something as well.

For predictions - it may not matter - it's possible to just assign a number to each category, regardless of order, and move along our merry ways. However, it's important in business to be able to interpret and explain model results, and you might not get to be there to walk through your arbitrary encoding scheme with The Powers That Be(tm).

In [10]:
# glue data sets together
train_test = pd.concat((train[features], test[features])).reset_index(drop=True)
# Convert categoricals to codes
for c in range(len(cat_features)):
    train_test[cat_features[c]] = train_test[cat_features[c]].astype('category').cat.codes

First the coding helper function, stolen from [here](https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/)

In [15]:
#Define a generic function using Pandas replace function
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

Now the coding:

In [21]:
train_test['ExterQual'] = coding(train_test['ExterQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['ExterCond'] = coding(train_test['ExterCond'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['BsmtQual'] = coding(train_test['BsmtQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['BsmtCond'] = coding(train_test['BsmtCond'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['BsmtExposure'] = coding(train_test['BsmtExposure'], {'Gd':4,'Av':3,'Mn':2,'No':1, 0:0})

In [22]:
train_test['HeatingQC'] = coding(train_test['HeatingQC'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['KitchenQual'] = coding(train_test['KitchenQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['FireplaceQu'] = coding(train_test['FireplaceQu'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['GarageQual'] = coding(train_test['GarageQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['GarageCond'] = coding(train_test['GarageCond'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['PoolQC'] = coding(train_test['PoolQC'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})

### 3.4 Restore training and test sets
I keep forgetting about iloc. It makes cutting up Pandas DataFrames work kind of like dataframes in R do. We'll put the training and test sets back in the same boxes as before.

In [28]:
trainX = train_test.iloc[:rows_train,:]
testX = train_test.iloc[rows_train:,:]

## 4 Data exploration
Now it's time to look at the data and see what, if anything, looks weird or notable.

### 4.1 Distribution of target variable
Let's look at the distribution of the target variable to see if we need to adjust it. Since RMSE of the logs is the standard by which we'll be measured, we'll probably have to log-transform the sale prices.